NEURAL NETWORK DESIGN

INPUT -> LAYER1 -> RELU -> LAYER2 -> SOFTMAX -> OUTPUTS/PROBABILITIES


In [1]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

In [2]:
from My_NN.nn import Layer_Dense
from My_NN.activation import ReLU, Softmax


In [3]:
x, y = spiral_data(samples=100, classes=3)

x.shape, y.shape

((300, 2), (300,))

In [4]:
dense1 = Layer_Dense(n_inputs=2, n_neurons=3)
activation1 = ReLU()
dense2 = Layer_Dense(n_inputs=3, n_neurons=3)
activation2 = Softmax()

In [5]:
dense1.forward(x)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
activation2.forward(dense2.output)

print(activation2.output[:5])

[[0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333288 0.33333366 0.33333346]
 [0.33333315 0.33333347 0.33333338]]


## Categorical Cross Entropy Loss FORMULA

```
loss = - sum{true-label * log(predicted-label)}
```

EXAMPLE

true: [1.0, 0.0, 0.0] predicted: [0.3, 0.6, 0.1]

sum(true-label * log(predicted-label)) = 1*log(0.3) + 0*log(0.6) + 0*log(0.1) = -1.736

loss = -sum = 1.736


In [6]:
import math
soft_max_output=[.7,.1,.2]
target_output=[1,0,0]
loss=-(target_output[0]*math.log(soft_max_output[0])+target_output[1]*math.log(soft_max_output[1])+target_output[2]*math.log(soft_max_output[2]))
print(loss)

0.35667494393873245


In [7]:
#using zip()
softmax_outputs = [[0.7, 0.1, 0.2],
[0.1, 0.5, 0.4],
[0.02, 0.9, 0.08]]
class_targets = [0, 1, 1]
for tar,dis in zip(class_targets,softmax_outputs):
  print(dis[tar])


0.7
0.5
0.9


In [8]:
#using numpy
softmax_outputs = np.array([
  [0.7, 0.1, 0.2],
  [0.1, 0.5, 0.4],
  [0.02, 0.9, 0.08]])
class_targets = [0, 1, 1]
print(softmax_outputs[range(len(softmax_outputs)), class_targets])
#Loss
neg_log=(-np.log(softmax_outputs[range(len(softmax_outputs)), class_targets]))
print(neg_log)
average_loss = np.mean(neg_log)
print(average_loss)

[0.7 0.5 0.9]
[0.35667494 0.69314718 0.10536052]
0.38506088005216804


In [9]:
#if class are one hot encode 
import numpy as np
softmax_outputs = np.array([
  [0.7, 0.1, 0.2],
  [0.1, 0.5, 0.4],
  [0.02, 0.9, 0.08]])
class_targets = np.array([
  [1, 0, 0],
  [0, 1, 0],
  [0, 1, 0]])
if len(class_targets.shape) == 1:
  correct_confidences = softmax_outputs[
  range(len(softmax_outputs)),
  class_targets
  ]
# Mask values - only for one-hot encoded labels
elif len(class_targets.shape) == 2:
  correct_confidences = np.sum(
  softmax_outputs * class_targets,
  axis=1
  )
# Losses
neg_log = -np.log(correct_confidences)
average_loss = np.mean(neg_log)
print(average_loss)
class_targets.ndim

0.38506088005216804


2

# CLASS IMPLEMENTATION OF CROSS_ENTROPY


In [10]:
class Loss:
  # Calculates the data and regularization losses
  # given model output and ground truth values
  def calculate(self, output, y):
    # Calculate sample losses
    sample_losses = self.forward(output, y)
    # Calculate mean loss
    data_loss = np.mean(sample_losses)
    # Return loss
    return data_loss

In [11]:
# Cross-entropy loss
class Loss_CategoricalCrossentropy(Loss):
    # Forward pass
    def forward(self, y_pred, y_true):
        
        y_pred = np.asarray(y_pred)
        y_true = np.asarray(y_true)
        
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)
        
        if len(y_true.shape)==1:
            correct_confidence = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape)==2:
            correct_confidence = np.sum(y_pred_clipped*y_true, axis=1)
        else:
            return np.nan
        
        negative_log_likelihood = -np.log(correct_confidence)
        return negative_log_likelihood
             


In [12]:
import numpy as np
softmax_outputs = np.array([
  [0.7, 0.1, 0.2],
  [0.1, 0.5, 0.4],
  [0.02, 0.9, 0.08]])
class_targets = np.array([
  [1, 0, 0],
  [0, 1, 0],
  [0, 1, 0]])

In [13]:
loss_function = Loss_CategoricalCrossentropy()
loss = loss_function.calculate(softmax_outputs, class_targets)
print(loss)



0.38506088005216804


# neural netowrk pipeline


In [14]:
x, y = spiral_data(samples=100, classes=3)

In [15]:
dense1 = Layer_Dense(n_inputs=2, n_neurons=3)
activation1 = ReLU()
dense2 = Layer_Dense(n_inputs=3, n_neurons=3)
activation2 = Softmax()

In [16]:
dense1.forward(x)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
activation2.forward(dense2.output)

print(activation2.output[:5])
loss = loss_function.calculate(activation2.output, y)
print('loss:', loss)


[[0.33333333 0.33333333 0.33333333]
 [0.3333333  0.33333353 0.33333317]
 [0.33333318 0.33333384 0.33333298]
 [0.33333308 0.33333412 0.3333328 ]
 [0.33333324 0.33333406 0.3333327 ]]
loss: 1.0986102365034156


## Accuracy


In [17]:
import numpy as np
# Probabilities of 3 samples
softmax_outputs = np.array([[0.7, 0.2, 0.1],
[0.5, 0.1, 0.4],
[0.02, 0.9, 0.08]])
# Target (ground-truth) labels for 3 samples
class_targets = np.array([0,0, 1])
# Calculate values along second axis (axis of index 1)
predictions = np.argmax(softmax_outputs, axis=1)
# If targets are one-hot encoded - convert them
if len(class_targets.shape) == 2:
  class_targets = np.argmax(class_targets, axis=1)  #returns the indices of the maximum values along an axis.
# True evaluates to 1; False to 0
accuracy = np.mean(predictions==class_targets)
print('acc:', accuracy)
predictions

acc: 1.0


array([0, 0, 1], dtype=int64)